In [90]:
import requests
import json
import pandas as pd
from io import StringIO

In [94]:
#Fields to query in the cases endpoint
cases_endpt = "https://api.gdc.cancer.gov/cases"
fields = [
    "project.project_id",
    "primary_site",
    "demographic.gender",
    "disease_type",
    "diagnoses.vital_status"
    ]
fields = ",".join(fields)

#Fields to query in the files endpoint

#Set of filters nested under an operator
filters = {
            "op":"in",
            "content":{
                "field": "files.analysis.workflow_type",
                "value": ["HTSeq - Counts"]
                }
        }

# With a GET request, the filters parameter needs to be converted
# from a dictionary to JSON-formatted string

params = {
    "filters": json.dumps(filters),
    "fields": fields,
    "format": "TSV",
    "size": 15000
    }

response_cases = requests.get(cases_endpt,params = params)

In [197]:
#Fields to query in the files endpoint
cases_endpt = "https://api.gdc.cancer.gov/files"
fields_files = [
    "file_name",
    "cases.samples.sample_type",
    "cases.disease_type",
    "cases.project.project_id",
    "cases.case_id",
    "cases.primary_site",
    "cases.demographic.gender",
    "cases.diagnoses.vital_status"
    ]
fields_files = ",".join(fields_files)

#Fields to query in the files endpoint

#Set of filters nested under an operator
filters = {
            "op":"in",
            "content":{
                "field": "files.analysis.workflow_type",
                "value": ["HTSeq - Counts"]
                }
        }

# With a GET request, the filters parameter needs to be converted
# from a dictionary to JSON-formatted string

params_files = {
    "filters": json.dumps(filters),
    "fields": fields_files,
    "format": "TSV",
    "size": 150000
    }

response_files = requests.get(files_endpt,params = params_files)

In [198]:
files = pd.read_table(StringIO(response_files.content.decode("utf-8")),sep="\t")
files.head()

,cases.0.samples.0.sample_type,file_name,cases.0.project.project_id,cases.0.primary_site,cases.0.diagnoses.0.vital_status,cases.0.disease_type,cases.0.demographic.gender,cases.0.case_id,id
0,Primary Tumor,81cab804-bfb1-44f0-ac2d-923e0766ac61.htseq.cou...,TCGA-SARC,Soft Tissue,alive,Sarcoma,male,8f3f6f7c-4b50-467e-a6e8-d836735913f6,0934a0b3-858d-430c-a5c9-ea459104a392
1,Primary Tumor,155f0f8e-9af5-452a-92ee-8b49d5df864e.htseq.cou...,TCGA-LGG,Brain,alive,Brain Lower Grade Glioma,female,887def39-e212-490d-8849-2bc932fceff0,e7643723-dbbc-4c6c-981a-f0ccccf2aa9f
2,Primary Tumor,49b5b772-6a7b-4aae-a9ed-8a95a0d9a28d.htseq.cou...,TCGA-THCA,Thyroid,alive,Thyroid Carcinoma,female,efbdedaa-4b02-40ac-b93c-9670c018cf7f,39a74fe1-e4e0-4118-8489-75aefb4735b3
3,Primary Tumor,f9dc7237-e186-40e8-ab4c-d142dba7cf3d.htseq.cou...,TCGA-OV,Ovary,alive,Ovarian Serous Cystadenocarcinoma,female,60014a6e-6a13-4f92-bbfd-eeeee9632f98,dd9bc896-11ad-4156-9fd2-e85be97507e4
4,Primary Tumor,86d7188b-2d97-4dca-b033-34778e5a6aa1.htseq.cou...,TCGA-MESO,Pleura,dead,Mesothelioma,male,92c1f122-2f48-47a7-b3b8-c1792a1f4f32,e1617a0f-a75d-471b-ad45-c78b78ba1615


In [216]:
only_lihc = files['cases.0.project.project_id'] == 'TCGA-LIHC'
only_healthy = files['cases.0.samples.0.sample_type'] != "Primary Tumor"
files[only_lihc].shape[0] - files[only_lihc & only_healthy].shape[0]

371

In [95]:
cases = pd.read_table(StringIO(response_cases.content.decode("utf-8")),sep="\t")

In [104]:
cases.head()

,project.project_id,disease_type,diagnoses.0.vital_status,primary_site,id,demographic.gender
0,TCGA-OV,Ovarian Serous Cystadenocarcinoma,dead,Ovary,dce71741-ccbe-40b7-a0b8-2048d07187a4,female
1,TCGA-THCA,Thyroid Carcinoma,alive,Thyroid,e8f56d0f-eee4-4def-a43a-dec91f4382a1,female
2,TARGET-NBL,Neuroblastoma,dead,Nervous System,657566a4-d809-53af-9acd-d44fa6171b48,female
3,TCGA-LGG,Brain Lower Grade Glioma,dead,Brain,b9e2f4d1-866c-4c62-a9c8-8de7eb364a6e,male
4,TCGA-SARC,Sarcoma,dead,Soft Tissue,6201394f-b70e-4f14-9519-ff9d5f06eb09,female


In [53]:
stnJson = json.dumps(response.json(), indent=2)
print(stnJson)

{
  "data": {
    "hits": [
      {
        "primary_site": "Ovary",
        "disease_type": "Ovarian Serous Cystadenocarcinoma",
        "demographic": {
          "gender": "female"
        },
        "id": "dce71741-ccbe-40b7-a0b8-2048d07187a4",
        "diagnoses": [
          {
            "vital_status": "dead"
          }
        ],
        "project": {
          "project_id": "TCGA-OV"
        }
      },
      {
        "primary_site": "Thyroid",
        "disease_type": "Thyroid Carcinoma",
        "demographic": {
          "gender": "female"
        },
        "id": "e8f56d0f-eee4-4def-a43a-dec91f4382a1",
        "diagnoses": [
          {
            "vital_status": "alive"
          }
        ],
        "project": {
          "project_id": "TCGA-THCA"
        }
      },
      {
        "primary_site": "Nervous System",
        "disease_type": "Neuroblastoma",
        "demographic": {
          "gender": "female"
        },
        "id": "657566a4-d809-53af-9acd-d44fa6171b48

In [57]:
response.content.decode("utf-8")

'{"data": {"hits": [{"project": {"project_id": "TCGA-OV"}, "diagnoses": [{"vital_status": "dead"}], "primary_site": "Ovary", "disease_type": "Ovarian Serous Cystadenocarcinoma", "demographic": {"gender": "female"}, "id": "dce71741-ccbe-40b7-a0b8-2048d07187a4"}, {"project": {"project_id": "TCGA-THCA"}, "diagnoses": [{"vital_status": "alive"}], "primary_site": "Thyroid", "disease_type": "Thyroid Carcinoma", "demographic": {"gender": "female"}, "id": "e8f56d0f-eee4-4def-a43a-dec91f4382a1"}, {"project": {"project_id": "TARGET-NBL"}, "diagnoses": [{"vital_status": "dead"}], "primary_site": "Nervous System", "disease_type": "Neuroblastoma", "demographic": {"gender": "female"}, "id": "657566a4-d809-53af-9acd-d44fa6171b48"}, {"project": {"project_id": "TCGA-LGG"}, "diagnoses": [{"vital_status": "dead"}], "primary_site": "Brain", "disease_type": "Brain Lower Grade Glioma", "demographic": {"gender": "male"}, "id": "b9e2f4d1-866c-4c62-a9c8-8de7eb364a6e"}, {"project": {"project_id": "TCGA-SARC"}, 

In [72]:
apple = response.content.decode("utf-8")

In [81]:
json.dumps(json.loads(apple)["data"]['hits'])

'[{"primary_site": "Ovary", "disease_type": "Ovarian Serous Cystadenocarcinoma", "demographic": {"gender": "female"}, "id": "dce71741-ccbe-40b7-a0b8-2048d07187a4", "diagnoses": [{"vital_status": "dead"}], "project": {"project_id": "TCGA-OV"}}, {"primary_site": "Thyroid", "disease_type": "Thyroid Carcinoma", "demographic": {"gender": "female"}, "id": "e8f56d0f-eee4-4def-a43a-dec91f4382a1", "diagnoses": [{"vital_status": "alive"}], "project": {"project_id": "TCGA-THCA"}}, {"primary_site": "Nervous System", "disease_type": "Neuroblastoma", "demographic": {"gender": "female"}, "id": "657566a4-d809-53af-9acd-d44fa6171b48", "diagnoses": [{"vital_status": "dead"}], "project": {"project_id": "TARGET-NBL"}}, {"primary_site": "Brain", "disease_type": "Brain Lower Grade Glioma", "demographic": {"gender": "male"}, "id": "b9e2f4d1-866c-4c62-a9c8-8de7eb364a6e", "diagnoses": [{"vital_status": "dead"}], "project": {"project_id": "TCGA-LGG"}}, {"primary_site": "Soft Tissue", "disease_type": "Sarcoma", 